# Training 

In [1]:
import datetime
import time
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import codecs
import pickle
import math

from model_partial_ner.ner import NER
import model_partial_ner.utils as utils
from model_partial_ner.object import softCE
from model_partial_ner.basic import BasicRNN
from model_partial_ner.dataset import NERDataset, TrainDataset

from torch_scope import wrapper

import argparse
import logging
import json
import os
import sys
import itertools
import functools

logger = logging.getLogger(__name__)

In [2]:
cp_root = "./models/custom/checkpoint/"
checkpoint_name = "autoner"

In [3]:
pw = wrapper(os.path.join(cp_root, checkpoint_name), 
             checkpoint_name, enable_git_track="store_true", seed = 234)

[2022-03-21 03:44:09,715] Saving system environemnt and python packages         
[2022-03-21 03:44:10,432] Setting up git tracker                                


In [4]:
gpu_index = pw.auto_device()
device = torch.device("cuda:" + str(gpu_index) if gpu_index >= 0 else "cpu")

[2022-03-21 03:45:23,822] It's recommended to set ``CUDA_DEVICE_ORDER``to be ``PCI_BUS_ID`` by ``export CUDA_DEVICE_ORDER=PCI_BUS_ID``;otherwise, it's not guaranteed that the gpu index frompytorch to be consistent the ``nvidia-smi`` results.
[2022-03-21 03:45:23,851] GPU memory usages:                                    
[2022-03-21 03:45:23,852] GPU ID: Mem	 Utils                                    
[2022-03-21 03:45:23,852] GPU  0: 3 MiB	  0 %                                   
[2022-03-21 03:45:23,853] Recommended GPU Index: 0                              


In [5]:
eval_dataset = "models/custom/encoded_data/test.pk"

In [6]:
logger.info('loading dataset')
key_list = ['emb_array', 'w_map', 'c_map', 'tl_map', 'cl_map', 'range', 'test_data', 'dev_data']
dataset = pickle.load(open(eval_dataset, 'rb'))


[2022-03-12 18:43:33,743] loading dataset                                       


In [7]:
dataset.keys()

dict_keys(['emb_array', 'w_map', 'c_map', 'tl_map', 'cl_map', 'range', 'test_data', 'dev_data'])

In [8]:
emb_array, w_map, c_map, tl_map, cl_map, range_idx, test_data, dev_data = [dataset[tup] for tup in key_list]
id2label = {v: k for k, v in tl_map.items()}

In [9]:
id2label

{0: 'None', 1: 'DISEASE'}

In [10]:
cl_map

{'I': 0, 'O': 1}

In [11]:
train_dataset = "models/custom/encoded_data/train_0.pk"
batch_token_number = 3000
sample_ratio = 1

In [12]:
assert len(emb_array) == len(w_map)

train_loader = TrainDataset(train_dataset, 
                            w_map['<\n>'], 
                            c_map['<\n>'], 
                            batch_token_number, 
                            sample_ratio = sample_ratio)

test_loader = NERDataset(
    test_data, w_map['<\n>'], 
    c_map['<\n>'], 
    batch_token_number)

dev_loader = NERDataset(
    dev_data, 
    w_map['<\n>'], 
    c_map['<\n>'],
    batch_token_number)

In [20]:
rnn_layer = 'Basic'
layer_num = 2
rnn_unit = 'lstm'
word_dim = 200
char_dim = 30
hid_dim = 300
droprate = 0.5
batch_norm = 'store_true'
lr = 0.05
label_dim = 50
update = 'SGD'
tolerance = 5

In [21]:
import numpy as np

In [22]:
rnn_map = {'Basic': BasicRNN}
rnn_layer = rnn_map[rnn_layer](
    layer_num, 
    rnn_unit, 
    word_dim + char_dim, 
    hid_dim, 
    droprate, 
    batch_norm)

ner_model = NER(rnn_layer, len(w_map), 
                word_dim, len(c_map), 
                char_dim, label_dim, 
                len(tl_map), droprate)

ner_model.rand_ini()
ner_model.load_pretrained_word_embedding(torch.FloatTensor(np.array(emb_array)))
ner_config = ner_model.to_params()
ner_model.to(device)

optim_map = {'Adam' : optim.Adam, 'Adagrad': optim.Adagrad, 'Adadelta': optim.Adadelta, 'SGD': functools.partial(optim.SGD, momentum=0.9)}
if lr > 0:
    optimizer=optim_map[update](ner_model.parameters(), lr=lr)
else:
    optimizer=optim_map[update](ner_model.parameters())

crit_chunk = nn.BCEWithLogitsLoss()
crit_type = softCE()

In [23]:
ner_model

NER(
  (rnn): BasicRNN(
    (layer): Sequential(
      (0): BasicUnit(
        (layer): LSTM(230, 150, batch_first=True, bidirectional=True)
        (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicUnit(
        (layer): LSTM(300, 150, batch_first=True, bidirectional=True)
        (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (word_embed): Embedding(3661, 200)
  (char_embed): Embedding(79, 30)
  (drop): Dropout(p=0.5, inplace=False)
  (to_chunk): highway(
    (trans): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (to_type): highway(
    (trans): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out

In [24]:
pw.save_configue({'w_map': w_map, 'c_map': c_map, 'tl_map': tl_map, 'cl_map': cl_map}, 'dict.json')

In [25]:
logger.info('Setting up training environ.')

batch_index = 0
best_eval = float('-inf')
best_f1, best_pre, best_rec = -1, -1, -1
best_type2f1, best_type2pre, best_type2rec = {}, {}, {}

patience = 0
current_lr = lr


[2022-03-12 18:45:28,455] Setting up training environ.                          


In [37]:
epoch = 50
clip = 5
interval = 30
check = 1000

In [38]:
for indexs in range(epoch):

    logger.info('############')
    logger.info('Epoch: {}'.format(indexs))
    ner_model.train()

    for word_t, char_t, chunk_mask, chunk_label, type_mask, type_label in train_loader.get_tqdm(device):
        ner_model.zero_grad()
        output = ner_model(word_t, char_t, chunk_mask)

        chunk_score = ner_model.chunking(output)
        chunk_loss = crit_chunk(chunk_score, chunk_label)

        type_score = ner_model.typing(output, type_mask)
        type_loss = crit_type(type_score, type_label)

        loss = type_loss + chunk_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(ner_model.parameters(), clip)
        optimizer.step()

        batch_index += 1 

        if 0 == batch_index % interval:
            pw.add_loss_vs_batch(
                {'loss_chunk': utils.to_scalar(chunk_loss),
                 'loss_type': utils.to_scalar(type_loss)}, 
                batch_index, 
                use_logger = False
            )


        if 0 == batch_index % check:

            # NER evaluation
            pre_dev, rec_dev, f1_dev, type2pre_dev, type2rec_dev, type2f1_dev = utils.evaluate_ner(
                dev_loader.get_tqdm(device), ner_model, tl_map['None'], id2label)
            pw.add_loss_vs_batch({'dev_pre': pre_dev, 'dev_rec': rec_dev}, batch_index, use_logger = False)
            pw.add_loss_vs_batch({'dev_f1': f1_dev}, batch_index, use_logger = True)

            pw.save_checkpoint(model = ner_model, is_best = f1_dev > best_eval, s_dict = {'config': ner_config, 'w_map': w_map, 'c_map': c_map, 'tl_map': tl_map, 'cl_map': cl_map})

            if f1_dev > best_eval:
                best_eval = f1_dev
                # best_f1, best_pre, best_rec, best_type2pre, best_type2rec, best_type2f1 = utils.evaluate_ner(test_loader.get_tqdm(device), ner_model, tl_map['None'])
                best_pre, best_rec, best_f1, best_type2pre, best_type2rec, best_type2f1 = utils.evaluate_ner(test_loader.get_tqdm(device), ner_model, tl_map['None'], id2label)
                pw.add_loss_vs_batch({'test_pre': best_pre, 'test_rec': best_rec}, batch_index, use_logger = False)
                pw.add_loss_vs_batch({'test_f1': best_f1}, batch_index, use_logger = True)
                patience = 0
                for entity_type in best_type2f1:
                    pw.add_loss_vs_batch({'per_{}_f1'.format(entity_type): best_type2f1[entity_type], 
                                            'per_{}_pre'.format(entity_type): best_type2pre[entity_type],
                                            'per_{}_rec'.format(entity_type): best_type2rec[entity_type]}, batch_index, use_logger = False)
                    logger.info('\ttype: %s, f1: %.6f, pre: %.6f, rec: %.6f' % (entity_type, best_type2f1[entity_type], best_type2pre[entity_type], best_type2rec[entity_type]))

            else:
                patience += 1
                if patience >= tolerance:
                    patience = 0
                    current_lr *= 0.9 ** tolerance
                    if args.update == 'SGD':
                        utils.adjust_learning_rate(optimizer, current_lr)
                        logger.info('current_lr = %.10f' % current_lr)

            ner_model.train()


[2022-03-12 18:52:10,982] ############                                          
[2022-03-12 18:52:10,983] Epoch: 0                                              


[2022-03-12 18:52:23,412] ############                                          
[2022-03-12 18:52:23,413] Epoch: 1                                              


[2022-03-12 18:52:35,886] ############                                          
[2022-03-12 18:52:35,887] Epoch: 2                                              


[2022-03-12 18:52:48,330] ############                                          
[2022-03-12 18:52:48,331] Epoch: 3                                              


[2022-03-12 18:53:00,763] ############                                          
[2022-03-12 18:53:00,764] Epoch: 4                                              


...

[2022-03-12 19:01:44,381] ############                                          
[2022-03-12 19:01:44,381] Epoch: 45                                             


[2022-03-12 19:01:56,879] ############                                          
[2022-03-12 19:01:56,880] Epoch: 46                                             


[2022-03-12 19:02:09,233] ############                                          
[2022-03-12 19:02:09,234] Epoch: 47                                             


[2022-03-12 19:02:21,705] ############                                          
[2022-03-12 19:02:21,706] Epoch: 48                                             


[2022-03-12 19:02:34,205] ############                                          
[2022-03-12 19:02:34,206] Epoch: 49                                             


In [39]:
# NER evaluation
pre_dev, rec_dev, f1_dev, type2pre_dev, type2rec_dev, type2f1_dev = utils.evaluate_ner(
    dev_loader.get_tqdm(device), ner_model, tl_map['None'], id2label
)

In [40]:
type2pre_dev

{'DISEASE': 0.8719999986048}

In [41]:
type2rec_dev

{'DISEASE': 0.8978583181254394}

In [42]:
type2f1_dev

{'DISEASE': 0.8847397584110073}

In [43]:
pw.add_loss_vs_batch({'dev_pre': pre_dev, 'dev_rec': rec_dev}, batch_index, use_logger = False)
pw.add_loss_vs_batch({'dev_f1': f1_dev}, batch_index, use_logger = True)
pw.save_checkpoint(model = ner_model, is_best = True, s_dict = {'config': ner_config, 'w_map': w_map, 'c_map': c_map, 'tl_map': tl_map, 'cl_map': cl_map})


[2022-03-12 19:18:31,294] dev_f1 : 0.8847397584110073                           


# Inference

In [44]:
from __future__ import print_function
import datetime
import time
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import codecs
import pickle
import math
import numpy as np

from model_partial_ner.ner import NER
from model_partial_ner.basic import BasicRNN
from model_partial_ner.dataset import RawDataset
import model_partial_ner.utils as utils

from torch_scope import basic_wrapper as bw

import argparse
import logging
import json
import os
import sys
import itertools
import functools

In [48]:
checkpoint_folder = "./models/custom/checkpoint/autoner/"
input_corpus = "./models/custom/encoded_test.pk"

In [46]:
logger.info('loading checkpoint')
checkpoint_file = bw.restore_best_checkpoint(checkpoint_folder)
w_map, c_map, tl_map, model = [checkpoint_file[name] for name in ['w_map', 'c_map', 'tl_map', 'model']]
id2label = {v: k for k, v in tl_map.items()}

In [47]:
id2label

{0: 'None', 1: 'DISEASE'}

In [49]:
logger.info('loading dataset')
raw_data = pickle.load(open(input_corpus, 'rb'))
data_loader = RawDataset(raw_data, w_map['<\n>'], c_map['<\n>'], batch_token_number)


[2022-03-12 19:57:35,147] loading dataset                                       


In [52]:
len(raw_data)

983

In [57]:
rnn_layer = 'Basic'

In [58]:
logger.info('building model')
rnn_map = {'Basic': BasicRNN}
rnn_layer = rnn_map[rnn_layer](
    layer_num, rnn_unit, 
    word_dim + char_dim, 
    hid_dim, droprate, 
    batch_norm)


[2022-03-12 20:00:34,237] building model                                        


In [59]:
ner_model = NER(rnn_layer, len(w_map), 
                word_dim, len(c_map), 
                char_dim, label_dim, 
                len(tl_map), droprate)
ner_model.load_state_dict(model)
ner_model.to(device)
ner_model.eval()

NER(
  (rnn): BasicRNN(
    (layer): Sequential(
      (0): BasicUnit(
        (layer): LSTM(230, 150, batch_first=True, bidirectional=True)
        (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicUnit(
        (layer): LSTM(300, 150, batch_first=True, bidirectional=True)
        (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (word_embed): Embedding(3661, 200)
  (char_embed): Embedding(79, 30)
  (drop): Dropout(p=0.5, inplace=False)
  (to_chunk): highway(
    (trans): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (to_type): highway(
    (trans): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (gate): ModuleList(
      (0): Linear(in_features=300, out

In [61]:
output_text = "./models/custom/decoded.txt"

In [63]:
output_list = list()

fout = open(output_text, 'w')

iterator = data_loader.get_tqdm(device)
max_score = -float('inf')
min_score = float('inf')


  0%|                                                                                             | 0/983 [00:00<?, ?it/s]

In [65]:
threshold = 0.0

In [66]:
for word_t, char_t, chunk_mask, chunk_index, chunk_surface in iterator:
    output = ner_model(word_t, char_t, chunk_mask)
    chunk_score = ner_model.chunking(output)

    tmp_min = utils.to_scalar(chunk_score.min())
    tmp_max = utils.to_scalar(chunk_score.max())
    max_score = max(max_score, tmp_max)
    min_score = min(min_score, tmp_min)

    pred_chunk = (chunk_score < threshold)

    chunk_index = chunk_index.masked_select(pred_chunk).data.cpu()

    output = ner_model.typing(output, pred_chunk)

    output = output.data.cpu()
    offset = chunk_index[0]
    for ind in range(0, output.size(0)):
        st, ed = chunk_index[ind].item(), chunk_index[ind + 1].item()
        surface = ' '.join(chunk_surface[st - offset : ed - offset])
        ent_type_id = np.argmax(output[ind]).item()
        ent_type = id2label[ent_type_id]

        values = [st, ed, surface, ent_type_id, ent_type]
        str_values = [str(v) for v in values]
        fout.write('\t'.join(str_values) + '\n')
        logger.info('\t'.join(str_values) + '\n')
    fout.write('\n')
    # logger.info('\n')

logger.info('max: '+str(max_score))
logger.info('min: '+str(min_score))
fout.close()

[2022-03-12 20:03:38,921] 11	12	Human	0	None
                                   
[2022-03-12 20:03:38,923] 12	13	iPSC	0	None
                                    
[2022-03-12 20:03:38,925] 13	14	-	0	None
                                       
[2022-03-12 20:03:38,926] 14	15	Derived	0	None
                                 
[2022-03-12 20:03:38,927] 15	16	Cerebellar	0	None
                              
[2022-03-12 20:03:38,928] 16	17	Neurons	0	None
                                 
[2022-03-12 20:03:38,930] 17	18	from	0	None
                                    
[2022-03-12 20:03:38,931] 18	19	a	0	None
                                       
[2022-03-12 20:03:38,932] 19	20	Patient	0	None
                                 
[2022-03-12 20:03:38,933] 20	21	with	0	None
                                    
[2022-03-12 20:03:38,934] 21	22	Ataxia	1	DISEASE
                               
[2022-03-12 20:03:38,935] 22	23	-	0	None
                                       
[2022-03-12 20:03:38,936] 23

[2022-03-12 20:03:39,225] 99	100	Liver	0	None
                                  
[2022-03-12 20:03:39,228] 100	101	gene	0	None
                                  
[2022-03-12 20:03:39,229] 101	102	expression	0	None
                            
[2022-03-12 20:03:39,231] 102	103	profiles	0	None
                              
[2022-03-12 20:03:39,232] 103	104	of	0	None
                                    
[2022-03-12 20:03:39,259] 105	106	interferon	0	None
                            
[2022-03-12 20:03:39,260] 106	107	therapy	0	None
                               
[2022-03-12 20:03:39,261] 107	108	in	0	None
                                    
[2022-03-12 20:03:39,262] 108	110	chronic hepatitis	1	DISEASE
                  
[2022-03-12 20:03:39,264] 110	111	B	0	None
                                     
[2022-03-12 20:03:39,265] 111	112	Patients	0	None
                              
[2022-03-12 20:03:39,266] 112	113	(	0	None
                                     
[2022-03-12 20:03:39,268] 11

[2022-03-12 20:03:39,589] 190	191	of	0	None
                                    
[2022-03-12 20:03:39,590] 191	192	peripheral	0	None
                            
[2022-03-12 20:03:39,592] 192	193	blood	0	None
                                 
[2022-03-12 20:03:39,593] 193	194	monocytes	0	None
                             
[2022-03-12 20:03:39,594] 194	195	in	0	None
                                    
[2022-03-12 20:03:39,595] 195	196	5	0	None
                                     
[2022-03-12 20:03:39,596] 196	197	non	0	None
                                   
[2022-03-12 20:03:39,597] 197	198	-	0	None
                                     
[2022-03-12 20:03:39,598] 198	199	smoking	0	None
                               
[2022-03-12 20:03:39,600] 199	200	moderate	0	None
                              
[2022-03-12 20:03:39,601] 200	201	to	0	None
                                    
[2022-03-12 20:03:39,602] 201	202	severe	0	None
                                
[2022-03-12 20:03:39,603] 20

[2022-03-12 20:03:40,139] 282	283	cell	0	None
                                  
[2022-03-12 20:03:40,140] 283	284	transcriptome	0	None
                         
[2022-03-12 20:03:40,141] 284	285	response	0	None
                              
[2022-03-12 20:03:40,143] 285	286	to	0	None
                                    
[2022-03-12 20:03:40,144] 286	287	Zaire	0	None
                                 
[2022-03-12 20:03:40,145] 287	288	Ebola	0	None
                                 
[2022-03-12 20:03:40,146] 288	289	virus	0	None
                                 
[2022-03-12 20:03:40,148] 289	290	wild	0	None
                                  
[2022-03-12 20:03:40,149] 290	291	-	0	None
                                     
[2022-03-12 20:03:40,150] 291	292	type	0	None
                                  
[2022-03-12 20:03:40,151] 292	293	in	0	None
                                    
[2022-03-12 20:03:40,153] 293	294	the	0	None
                                   
[2022-03-12 20:03:40,154] 29

[2022-03-12 20:03:40,500] 373	374	of	0	None
                                    
[2022-03-12 20:03:40,502] 374	375	interest	0	None
                              
[2022-03-12 20:03:40,502] 375	376	48	0	None
                                    
[2022-03-12 20:03:40,503] 376	377	hours	0	None
                                 
[2022-03-12 20:03:40,504] 377	378	prior	0	None
                                 
[2022-03-12 20:03:40,506] 378	379	to	0	None
                                    
[2022-03-12 20:03:40,507] 379	380	harvest	0	None
                               
[2022-03-12 20:03:40,555] 381	382	Liver	0	None
                                 
[2022-03-12 20:03:40,557] 382	383	transcriptome	0	None
                         
[2022-03-12 20:03:40,558] 383	384	profiles	0	None
                              
[2022-03-12 20:03:40,559] 384	385	correlates	0	None
                            
[2022-03-12 20:03:40,561] 385	386	with	0	None
                                  
[2022-03-12 20:03:40,562] 38

...

[2022-03-12 20:04:43,514] 15402	15403	and	0	None
                               
[2022-03-12 20:04:43,515] 15403	15404	COPD	1	DISEASE
                           
[2022-03-12 20:04:43,516] 15404	15405	subjects	0	None
                          
[2022-03-12 20:04:43,518] 15405	15406	before	0	None
                            
[2022-03-12 20:04:43,519] 15406	15407	and	0	None
                               
[2022-03-12 20:04:43,520] 15407	15408	after	0	None
                             
[2022-03-12 20:04:43,522] 15408	15409	3	0	None
                                 
[2022-03-12 20:04:43,523] 15409	15410	months	0	None
                            
[2022-03-12 20:04:43,524] 15410	15411	of	0	None
                                
[2022-03-12 20:04:43,525] 15411	15412	training	0	None
                          
[2022-03-12 20:04:43,581] 15413	15414	Transcription	0	None
                     
[2022-03-12 20:04:43,582] 15414	15415	profiling	0	None
                         
[2022-03-12 20:04:43,583] 15

[2022-03-12 20:04:43,875] 15492	15493	treated	0	None
                           
[2022-03-12 20:04:43,876] 15493	15494	with	0	None
                              
[2022-03-12 20:04:43,877] 15494	15495	valproate	0	None
                         
[2022-03-12 20:04:43,931] 15496	15497	Transcription	0	None
                     
[2022-03-12 20:04:43,933] 15497	15498	profiling	0	None
                         
[2022-03-12 20:04:43,935] 15498	15499	of	0	None
                                
[2022-03-12 20:04:43,936] 15499	15500	esophagus	0	None
                         
[2022-03-12 20:04:43,937] 15500	15501	samples	0	None
                           
[2022-03-12 20:04:43,938] 15501	15502	from	0	None
                              
[2022-03-12 20:04:43,939] 15502	15503	eight	0	None
                             
[2022-03-12 20:04:43,940] 15503	15504	human	0	None
                             
[2022-03-12 20:04:43,941] 15504	15505	patients	0	None
                          
[2022-03-12 20:04:43,943] 15

[2022-03-12 20:04:44,259] 15581	15582	of	0	None
                                
[2022-03-12 20:04:44,260] 15582	15583	immobilization	0	None
                    
[2022-03-12 20:04:44,318] 15584	15585	Transcription	0	None
                     
[2022-03-12 20:04:44,319] 15585	15586	profiling	0	None
                         
[2022-03-12 20:04:44,321] 15586	15587	of	0	None
                                
[2022-03-12 20:04:44,322] 15587	15588	human	0	None
                             
[2022-03-12 20:04:44,323] 15588	15590	prostate cancer	1	DISEASE
                
[2022-03-12 20:04:44,324] 15590	15591	cells	0	None
                             
[2022-03-12 20:04:44,326] 15591	15592	with	0	None
                              
[2022-03-12 20:04:44,327] 15592	15593	arachidonic	0	None
                       
[2022-03-12 20:04:44,328] 15593	15594	acid	0	None
                              
[2022-03-12 20:04:44,329] 15594	15595	reveals	0	None
                           
[2022-03-12 20:04:44,330] 15

[2022-03-12 20:04:44,684] 15674	15675	-	0	None
                                 
[2022-03-12 20:04:44,685] 15675	15676	lesional	0	None
                          
[2022-03-12 20:04:44,687] 15676	15677	skin	0	None
                              
[2022-03-12 20:04:44,688] 15677	15678	from	0	None
                              
[2022-03-12 20:04:44,689] 15678	15679	actopic	0	None
                           
[2022-03-12 20:04:44,690] 15679	15680	dermititis	0	None
                        
[2022-03-12 20:04:44,692] 15680	15681	patients	0	None
                          
[2022-03-12 20:04:44,693] 15681	15682	and	0	None
                               
[2022-03-12 20:04:44,695] 15682	15683	normal	0	None
                            
[2022-03-12 20:04:44,696] 15683	15684	skin	0	None
                              
[2022-03-12 20:04:44,747] 15685	15686	Expression	0	None
                        
[2022-03-12 20:04:44,749] 15686	15687	data	0	None
                              
[2022-03-12 20:04:44,751] 15

[2022-03-12 20:04:45,145] 15764	15765	to	0	None
                                
[2022-03-12 20:04:45,146] 15765	15766	characterise	0	None
                      
[2022-03-12 20:04:45,148] 15766	15767	RA	0	None
                                
[2022-03-12 20:04:45,149] 15767	15768	at	0	None
                                
[2022-03-12 20:04:45,150] 15768	15769	the	0	None
                               
[2022-03-12 20:04:45,151] 15769	15770	molecular	0	None
                         
[2022-03-12 20:04:45,153] 15770	15771	level	0	None
                             
[2022-03-12 20:04:45,154] 15771	15772	and	0	None
                               
[2022-03-12 20:04:45,155] 15772	15773	to	0	None
                                
[2022-03-12 20:04:45,156] 15773	15774	uncover	0	None
                           
[2022-03-12 20:04:45,158] 15774	15775	key	0	None
                               
[2022-03-12 20:04:45,159] 15775	15776	pathomechanisms	0	None
                   
[2022-03-12 20:04:45,228] 15